In [57]:
import numpy as np
import pandas as pd
import os
import csv, json
import math
import re
from PIL import Image

from collections import Counter

In [58]:
dataset_path = [
    '../dataset/validation_original(csv)/powershell.csv',
    '../dataset/validation_original(csv)/sample.csv',
]

In [ ]:
import re
#A = re.compile(r'')

def rc_block(code):
    return re.sub(r'<#.*?#>', '', code, flags=re.DOTALL)

def rc_line(code):
    return re.sub(r'#.*', '', code)

In [60]:
import re

def splitregex(text, pattern):
    pos = 0
    lastpos = len(text)
    splited = []
    for m in re.finditer(pattern, text):
        if pos < m.start():
            splited.append(text[pos:m.start()])
        splited.append(m.group())
        pos = m.end()
    if pos < lastpos:
        splited.append(text[pos:lastpos])
    return splited

RE_PS = re.compile(r'@"|@\'|"|\'|<#|#>|#|\n')
def rc_full(text):
    ignore_until = None
    discard_until = None
    group = splitregex(text, RE_PS)

    remain_text = []
    for g in group:
        if ignore_until is not None:
            if g == ignore_until:
                ignore_until = None
            remain_text.append(g)
        elif discard_until is not None:
            if g == discard_until:
                discard_until = None
        else:
            match g:
                case '"':
                    ignore_until = '"'
                    remain_text.append(g)
                case "'":
                    ignore_until = "'"
                    remain_text.append(g)
                case '<#':
                    discard_until = '#>'
                case '#':
                    discard_until = '\n'
                case _:
                    remain_text.append(g)
    return ''.join(remain_text)


In [67]:
def proc_rawcode(rawcode:str, hint):
    code = rc_full(rawcode)
    if len(code) != len(rawcode):
        print(f'#{hint['index']} {hint['row']['filename']}')
    return code

In [68]:
for filename in dataset_path:
    print(f'Processing {filename}...')
    dataset = pd.read_csv(filename)
    rows = dataset['bytes'][:]
    
    output_rows = []
    for i, row in enumerate(rows):
        rawcode = bytes(json.loads(row)).decode('utf-8')
        processed = proc_rawcode(rawcode, { 'index': i, 'row': dataset.iloc[i] })
        processed_code = json.dumps(list(processed.encode('utf-8')))
        output_rows.append({
            'path' : dataset['path'][i],
            'filename' : dataset['filename'][i],
            'bytes': processed_code,
        })
    output_df = pd.DataFrame(output_rows)

    output_dirname, output_filename = os.path.split(filename)
    
    output_path = f'{output_dirname}/rc_{output_filename}'
    output_df.to_csv(output_path, index=False)
    

Processing ../dataset/validation_original(csv)/powershell.csv...
#24 #_130_195bc776faf3a7509fbc9eaadd627e644907cf9a5b97510f495a12722dd56def.ps1
#29 #_136_1a1060a13985a9b8ccf9b51751d9de6ef969b5f34955735571162f654320b846.ps1
#93 #_20_03c3b71b2ad10276aa9e428e0c63bd3574131b85df8717607aae252a67aad0d7.ps1
#103 #_223_21c3648528da1269e5c07953ecda47853aa30cd49b82a03df3972e8fad958291.ps1
#104 #_224_21c5234439376dc4caecb30016b0877ef4479301681a10ae1d9bd75060d947e4.ps1
#118 #_243_22a8b4a7c7a467ea7fcf0a3930c99ecb482095093839683b400f58e2cdda176f.ps1
#153 #_27_04d66f2a208b0cdf66c41892f80618fc5142ec99e8314745e6cc7078552737f1.ps1
#192 #_323_282e79b30bad1c26ba96ebe6591b12788f8f5f892b07133f1c50ce1be1c65207.ps1
#204 #_342_29d58416b991f68e525ac735b73b06c4eea306d71542d79936c4851744dd5ac3.ps1
#216 #_35_071f027f09d3855e4553b589f22491c18876787377ecb305e8892749cc4364d4.ps1
#249 #_412_2d877b8166093c432785435e15c721f3a5c98351812971b430bcf8ab87e4c5a0.ps1
#332 #_531_35a939664efa46d1a7249b0a98538c6ee333a0be34b4ab6ffd